## Import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

## Load data

In [ ]:
# images = []
# ages = []
# genders = []
#
# for i in os.listdir('E:\\Pycharm\\Age-Detection-Using-Neural-Network\\DATA\\UTKFace\\Face_Detection\\'):
#     split = i.split('_')
#     ages.append(int(split[0]))
#     genders.append(int(split[1]))
#     images.append(Image.open('E:\\Pycharm\\Age-Detection-Using-Neural-Network\\DATA\\UTKFace\\Face_Detection\\' + i))


from PIL import Image
import os

images = []
ages = []
genders = []

folder = 'E:\\Pycharm\\Age-Detection-Using-Neural-Network\\DATA\\UTKFace\\Face_Detection\\'
files = os.listdir(folder)

for filename in files:
    split = filename.split('_')
    if len(split) >= 2 and split[0].isdigit() and split[1].isdigit():
        try:
            age = int(split[0])
            gender = int(split[1])
            path = os.path.join(folder, filename)

            with Image.open(path) as img:
                # img = img.resize((200, 200), Image.Resampling.LANCZOS)  # Resize nếu cần
                images.append(img.copy())  # copy để giữ ảnh trong RAM sau khi file đóng

            ages.append(age)
            genders.append(gender)
        except Exception as e:
            print(f"Bỏ qua ảnh lỗi {filename}: {e}")


In [ ]:
images = pd.Series(list(images), name='Images')
ages = pd.Series(list(ages), name='Ages')
genders = pd.Series(list(genders), name='Genders')

df = pd.concat([images, ages, genders], axis=1)
df

In [ ]:
display(df['Images'][0])
print(df['Ages'][0], df['Genders'][0])

In [ ]:
display(df['Images'][1])
print(df['Ages'][1], df['Genders'][1])

## Visualize

In [ ]:
sns.countplot(x='Ages', data=df)

Too many faces of people between 0 and 4 years old. The model would fit too well to these ages and not enough to the other ages. To resolve this I'm only going to include a third of the images between these ages.

In [ ]:
under4s = []

for i in range(len(df)):
    if df['Ages'].iloc[i] <= 4:
        under4s.append(df.iloc[i])
under4s = pd.DataFrame(under4s)
under4s = under4s.sample(frac=0.3)

df = df[df['Ages'] > 4]

df = pd.concat([df, under4s], ignore_index=True)

In [ ]:
sns.countplot(x='Ages', data=df)


In [ ]:
x = []
y = []

for i in range(len(df)):
    df['Images'].iloc[i] = df['Images'].iloc[i].resize((200, 200), Image.Resampling.LANCZOS)
    ar = np.asarray(df['Images'].iloc[i])
    x.append(ar)
    agegen = [int(df['Ages'].iloc[i]), int(df['Genders'].iloc[i])]
    y.append(agegen)
x = np.array(x)

## Train and Test

In [ ]:
y_age = df['Ages']
y_gender = df['Genders']

# x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(x, y_age, test_size=0.2, stratify=y_age)
# x_train_gender, x_test_gender, y_train_gender, y_test_gender = train_test_split(x, y_gender, test_size=0.2,
#                                                                                 stratify=y_gender)

x_train, x_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    x, y_age, y_gender, test_size=0.2, random_state=42
)

#
#
# y_age = df['Ages']
# y_gender = df['Genders']
#
# # Split for age prediction (no stratify)
# x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(
#     x, y_age, test_size=0.2, random_state=42
# )
#
# # Split for gender prediction (with stratify)
# x_train_gender, x_test_gender, y_train_gender, y_test_gender = train_test_split(
#     x, y_gender, test_size=0.2, stratify=y_gender, random_state=42
# )


## Model

In [ ]:
agemodel = Sequential()
agemodel.add(Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
agemodel.add(MaxPooling2D((2, 2)))
agemodel.add(Conv2D(64, (3, 3), activation='relu'))
agemodel.add(MaxPooling2D((2, 2)))
agemodel.add(Conv2D(128, (3, 3), activation='relu'))
agemodel.add(MaxPooling2D((2, 2)))
agemodel.add(Flatten())
agemodel.add(Dense(64, activation='relu'))
agemodel.add(Dropout(0.5))
agemodel.add(Dense(1, activation='relu'))

agemodel.compile(loss='mean_squared_error',
                 optimizer=optimizers.Adam(lr=0.0001))

genmodel = Sequential()
genmodel.add(Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
genmodel.add(MaxPooling2D((2, 2)))
genmodel.add(Conv2D(64, (3, 3), activation='relu'))
genmodel.add(MaxPooling2D((2, 2)))
genmodel.add(Conv2D(128, (3, 3), activation='relu'))
genmodel.add(MaxPooling2D((2, 2)))
genmodel.add(Flatten())
genmodel.add(Dense(64, activation='relu'))
genmodel.add(Dropout(0.5))
genmodel.add(Dense(1, activation='sigmoid'))

genmodel.compile(loss='binary_crossentropy',
                 optimizer=optimizers.Adam(lr=0.0001),
                 metrics=['accuracy'])

## Traing Model

In [ ]:
datagen = ImageDataGenerator(
    rescale=1. / 255., width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train1 = datagen.flow(x_train_age, y_train_age, batch_size=32)

test1 = test_datagen.flow(
    x_test_age, y_test_age,
    batch_size=32)

history1 = agemodel.fit(train1, epochs=50, shuffle=True, validation_data=test1)

In [ ]:
datagen = ImageDataGenerator(
      rescale=1./255., width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

train2 = datagen.flow(x_train_gender, y_train_gender, batch_size=64)

test2 = test_datagen.flow(
        x_test_gender, y_test_gender,
        batch_size=64)

history2 = genmodel.fit(train2, epochs=50, shuffle=True, validation_data=test2)

## Evaluate Model

In [ ]:
def process_and_predict(file):
    im = Image.open(file)
    width, height = im.size
    if width == height:
        im = im.resize((200,200), Image.Resampling.LANCZOS)
    else:
        if width > height:
            left = width/2 - height/2
            right = width/2 + height/2
            top = 0
            bottom = height
            im = im.crop((left,top,right,bottom))
            im = im.resize((200,200), Image.Resampling.LANCZOS)
        else:
            left = 0
            right = width
            top = 0
            bottom = width
            im = im.crop((left,top,right,bottom))
            im = im.resize((200,200), Image.Resampling.LANCZOS)

    ar = np.asarray(im)
    ar = ar.astype('float32')
    ar /= 255.0
    ar = ar.reshape(-1, 200, 200, 3)

    age = agemodel.predict(ar)
    gender = np.round(genmodel.predict(ar))
    if gender == 0:
        gender = 'male'
    elif gender == 1:
        gender = 'female'

    print('Age:', int(age), '\n Gender:', gender)
    return im.resize((300,300), Image.Resampling.LANCZOS)

In [ ]:
process_and_predict('E:\\Pycharm\\Age-Detection-Using-Neural-Network\\DATA\\UTKFace\\Face_Detection\\42_1_1_20170113005434508_face0.jpg')
